In [2]:
%pip install llama-index-llms-openai
%pip install llama-index-graph-stores-neo4j
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-azure-openai
%pip install llama-index-readers-file
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
llm = OpenAI(temperature=0, model="gpt-4o")
Settings.llm = llm
Settings.chunk_size = 512

In [5]:
documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
#neo4j creds
username = "neo4j"
password = ""
url = ""
database = "neo4j"

In [ ]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    show_progress=True,
    include_embeddings=True,
)

In [ ]:
query_engine = index.as_query_engine(
    include_text=True,
    embedding_mode="similarity",
    similarity_top_k=10,
    verbose=True
)

response = query_engine.query("Enter GRAG Question here")

In [ ]:
display(Markdown(f"{response}"))

In [ ]:
# OPTIONAL! Clear dataset from Neo4J store
graph_store.query(
    """
MATCH (n) DETACH DELETE n
"""
)